In [1]:
import pandas as pd

In [2]:
data_mail = pd.read_csv('C:\\Users\\Antonio\\Documents\\Modelos\\MailDataSet.csv',encoding='latin1')

In [3]:
data_mail.shape

(8266, 48)

In [4]:
data_mail

,bodydearword.feature,bodyform.feature,bodyhtml.feature,bodymultipart.feature,bodynumchars.feature,bodynumfunctionwords.feature,bodynumuniqwords.feature,bodynumwords.feature,bodyrichness.feature,bodysuspensionword.feature,...,urlnumperiods.feature,urlnumport.feature,urlport.feature,urltwodoains.feature,urlunmodalbaglink.feature,urlwordclicklink.feature,urlwordherelink.feature,urlwordloginlink.feature,urlwordupdatelink.feature,class
0,0,0,0,0,4522,0,374,931,0.205882,0,...,0,0,0,0,0,0,0,0,0,ham
1,0,0,0,0,890,1,124,198,0.222472,0,...,0,0,0,0,0,0,0,0,0,ham
2,0,0,0,0,3931,12,499,864,0.219791,0,...,0,0,0,1,0,0,0,0,0,ham
3,0,0,0,0,645,0,103,149,0.231008,0,...,0,0,0,0,0,0,0,0,0,ham
4,0,0,0,0,471,0,76,97,0.205945,0,...,0,0,0,0,0,0,0,0,0,ham
5,1,0,0,0,10691,20,760,2093,0.195772,0,...,0,0,0,0,0,0,0,0,0,ham
6,0,0,0,0,939,0,157,214,0.227902,0,...,0,0,0,0,0,0,0,0,0,ham
7,0,0,0,0,230,0,40,47,0.204348,0,...,0,0,0,0,0,0,0,0,0,ham
8,0,0,0,0,1187,9,153,238,0.200505,0,...,0,0,0,0,0,0,0,0,0,ham
9,0,0,0,0,4128,3,403,827,0.200339,0,...,0,0,0,0,0,0,0,0,0,ham


In [84]:
n_comp = 15

In [70]:
from sklearn.decomposition import PCA, FastICA, TruncatedSVD
pca = PCA(n_components=n_comp, random_state=420)
pca2_results_train = pca.fit_transform(data_mail.drop(["class"], axis=1))

In [71]:
from sklearn.decomposition import PCA, FastICA, TruncatedSVD
ica = FastICA(n_components=n_comp, random_state=100)
ica2_results_train = ica.fit_transform(data_mail.drop(["class"], axis=1))

In [72]:
from sklearn.decomposition import TruncatedSVD
tsvd = TruncatedSVD(n_components=n_comp, random_state=420)
tsvd_results_train = tsvd.fit_transform(data_mail.drop(["class"], axis=1))

In [73]:
from sklearn.random_projection import SparseRandomProjection
srp = SparseRandomProjection(n_components=n_comp, dense_output=True, random_state=420)
srp_results_train = srp.fit_transform(data_mail.drop(["class"], axis=1))

In [74]:
from sklearn.random_projection import GaussianRandomProjection
grp = GaussianRandomProjection(n_components=n_comp, eps=0.1, random_state=420)
grp_results_train = grp.fit_transform(data_mail.drop(["class"], axis=1))

In [33]:
def f(row):
    return 1 if row['class']=='ham' else 0

data_mail["class_mail"] = data_mail.apply(f, axis=1) # recorriendo columnas

In [75]:
data_mail_f = pd.DataFrame(data_mail["class_mail"])

In [76]:
for i in range(1, n_comp + 1):
    data_mail_f['pca_' + str(i)] = pca2_results_train[:, i - 1]
    data_mail_f['ica_' + str(i)] = ica2_results_train[:, i - 1]
    data_mail_f['tsvd_' + str(i)] = tsvd_results_train[:, i - 1]
    data_mail_f['srp_' + str(i)] = srp_results_train[:, i - 1]
    data_mail_f['grp_' + str(i)] = grp_results_train[:, i - 1]

In [77]:
no_contar = ['class_mail']
features = [x for x in data_mail_f.columns if x not in no_contar ]

In [78]:
target = ['class_mail']

In [79]:
features = [x for x in data_mail_f.columns if x not in target ]

In [80]:
X = data_mail_f[features].values
y = data_mail_f[target].values.ravel()

In [81]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=5678)

In [82]:
import lightgbm as lgb

train_set = lgb.Dataset(X_train, y_train)
test_sets = lgb.Dataset(X_test, y_test, reference=train_set)

In [83]:
params = {
    'task': 'train',
    'boosting_type': 'dart',
    'objective': 'binary',
    'metric': { 'mae' },
    'num_leaves': 64,
    #'min_data_in_leaf': 30,
    'learning_rate': 0.02,
    'max_depth': 6,
    'feature_fraction': 0.8,
    'bagging_freq': 1,
    'bagging_fraction': 0.8,
    'use_missing': False,
    #'is_unbalance': True,
    'verbose': 1
    }

import time
start_time = time.clock()
    
model = lgb.train(
    params,
    train_set,
    num_boost_round=15000,
    valid_sets=test_sets,
    early_stopping_rounds=50,
    #categorical_feature=index_categorical,
    verbose_eval=1)

[1]	valid_0's l1: 0.490421
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's l1: 0.481045
[3]	valid_0's l1: 0.471886
[4]	valid_0's l1: 0.462796
[5]	valid_0's l1: 0.453973
[6]	valid_0's l1: 0.445272
[7]	valid_0's l1: 0.436837
[8]	valid_0's l1: 0.439937
[9]	valid_0's l1: 0.431569
[10]	valid_0's l1: 0.423288
[11]	valid_0's l1: 0.415279
[12]	valid_0's l1: 0.418092
[13]	valid_0's l1: 0.410162
[14]	valid_0's l1: 0.40234
[15]	valid_0's l1: 0.394728
[16]	valid_0's l1: 0.387316
[17]	valid_0's l1: 0.380014
[18]	valid_0's l1: 0.372876
[19]	valid_0's l1: 0.365833
[20]	valid_0's l1: 0.358872
[21]	valid_0's l1: 0.360684
[22]	valid_0's l1: 0.353892
[23]	valid_0's l1: 0.347215
[24]	valid_0's l1: 0.340712
[25]	valid_0's l1: 0.334378
[26]	valid_0's l1: 0.328085
[27]	valid_0's l1: 0.321924
[28]	valid_0's l1: 0.324178
[29]	valid_0's l1: 0.318023
[30]	valid_0's l1: 0.312094
[31]	valid_0's l1: 0.312416
[32]	valid_0's l1: 0.306553
[33]	valid_0's l1: 0.300855
[34]	valid_0's l1: 0.295

[295]	valid_0's l1: 0.0851438
[296]	valid_0's l1: 0.0861053
[297]	valid_0's l1: 0.0845976
[298]	valid_0's l1: 0.0854792
[299]	valid_0's l1: 0.0840142
[300]	valid_0's l1: 0.0848458
[301]	valid_0's l1: 0.0858145
[302]	valid_0's l1: 0.0843433
[303]	valid_0's l1: 0.0852613
[304]	valid_0's l1: 0.0861819
[305]	valid_0's l1: 0.0871722
[306]	valid_0's l1: 0.0880456
[307]	valid_0's l1: 0.0888282
[308]	valid_0's l1: 0.0872724
[309]	valid_0's l1: 0.0881251
[310]	valid_0's l1: 0.0865798
[311]	valid_0's l1: 0.0850643
[312]	valid_0's l1: 0.0835683
[313]	valid_0's l1: 0.0821023
[314]	valid_0's l1: 0.0828729
[315]	valid_0's l1: 0.0837754
[316]	valid_0's l1: 0.0823911
[317]	valid_0's l1: 0.0809663
[318]	valid_0's l1: 0.0795487
[319]	valid_0's l1: 0.0803407
[320]	valid_0's l1: 0.0789456
[321]	valid_0's l1: 0.0796607
[322]	valid_0's l1: 0.0783028
[323]	valid_0's l1: 0.0769636
[324]	valid_0's l1: 0.0778263
[325]	valid_0's l1: 0.0764785
[326]	valid_0's l1: 0.0773274
[327]	valid_0's l1: 0.0759866
[328]	vali

[573]	valid_0's l1: 0.0217395
[574]	valid_0's l1: 0.0219276
[575]	valid_0's l1: 0.0216391
[576]	valid_0's l1: 0.0218348
[577]	valid_0's l1: 0.0220056
[578]	valid_0's l1: 0.0221924
[579]	valid_0's l1: 0.0223614
[580]	valid_0's l1: 0.0225465
[581]	valid_0's l1: 0.0222267
[582]	valid_0's l1: 0.0223961
[583]	valid_0's l1: 0.0221043
[584]	valid_0's l1: 0.0222872
[585]	valid_0's l1: 0.0219916
[586]	valid_0's l1: 0.0221467
[587]	valid_0's l1: 0.0218491
[588]	valid_0's l1: 0.0220285
[589]	valid_0's l1: 0.0222256
[590]	valid_0's l1: 0.0223745
[591]	valid_0's l1: 0.0225733
[592]	valid_0's l1: 0.0227601
[593]	valid_0's l1: 0.0224576
[594]	valid_0's l1: 0.0226269
[595]	valid_0's l1: 0.0227866
[596]	valid_0's l1: 0.022498
[597]	valid_0's l1: 0.0226734
[598]	valid_0's l1: 0.0228526
[599]	valid_0's l1: 0.0225385
[600]	valid_0's l1: 0.0222172
[601]	valid_0's l1: 0.021925
[602]	valid_0's l1: 0.0220865
[603]	valid_0's l1: 0.0217606
[604]	valid_0's l1: 0.0219257
[605]	valid_0's l1: 0.0220997
[606]	valid_

[854]	valid_0's l1: 0.00950321
[855]	valid_0's l1: 0.009409
[856]	valid_0's l1: 0.00933514
[857]	valid_0's l1: 0.00937037
[858]	valid_0's l1: 0.00940783
[859]	valid_0's l1: 0.00944771
[860]	valid_0's l1: 0.00936033
[861]	valid_0's l1: 0.00928929
[862]	valid_0's l1: 0.0092122
[863]	valid_0's l1: 0.00924607
[864]	valid_0's l1: 0.00928743
[865]	valid_0's l1: 0.00918643
[866]	valid_0's l1: 0.009095
[867]	valid_0's l1: 0.00903294
[868]	valid_0's l1: 0.00907051
[869]	valid_0's l1: 0.00898844
[870]	valid_0's l1: 0.00903018
[871]	valid_0's l1: 0.0090662
[872]	valid_0's l1: 0.00900261
[873]	valid_0's l1: 0.00904385
[874]	valid_0's l1: 0.00907803
[875]	valid_0's l1: 0.00911882
[876]	valid_0's l1: 0.00902845
[877]	valid_0's l1: 0.0090598
[878]	valid_0's l1: 0.0089684
[879]	valid_0's l1: 0.0090037
[880]	valid_0's l1: 0.00905026
[881]	valid_0's l1: 0.00909061
[882]	valid_0's l1: 0.00901413
[883]	valid_0's l1: 0.00904807
[884]	valid_0's l1: 0.0089705
[885]	valid_0's l1: 0.00888707
[886]	valid_0's l1

[1122]	valid_0's l1: 0.0059953
[1123]	valid_0's l1: 0.00596766
[1124]	valid_0's l1: 0.00598026
[1125]	valid_0's l1: 0.00594546
[1126]	valid_0's l1: 0.00590959
[1127]	valid_0's l1: 0.00592461
[1128]	valid_0's l1: 0.00590306
[1129]	valid_0's l1: 0.00588588
[1130]	valid_0's l1: 0.00589597
[1131]	valid_0's l1: 0.00590737
[1132]	valid_0's l1: 0.0058564
[1133]	valid_0's l1: 0.00582258
[1134]	valid_0's l1: 0.00580379
[1135]	valid_0's l1: 0.00576911
[1136]	valid_0's l1: 0.00578078
[1137]	valid_0's l1: 0.0057955
[1138]	valid_0's l1: 0.00580669
[1139]	valid_0's l1: 0.00581756
[1140]	valid_0's l1: 0.00577401
[1141]	valid_0's l1: 0.00578762
[1142]	valid_0's l1: 0.00579924
[1143]	valid_0's l1: 0.00576929
[1144]	valid_0's l1: 0.00574968
[1145]	valid_0's l1: 0.00572167
[1146]	valid_0's l1: 0.00573362
[1147]	valid_0's l1: 0.00570488
[1148]	valid_0's l1: 0.00571804
[1149]	valid_0's l1: 0.00569198
[1150]	valid_0's l1: 0.00570455
[1151]	valid_0's l1: 0.00571664
[1152]	valid_0's l1: 0.00572939
[1153]	vali

[1383]	valid_0's l1: 0.00456739
[1384]	valid_0's l1: 0.00457217
[1385]	valid_0's l1: 0.00457556
[1386]	valid_0's l1: 0.00458181
[1387]	valid_0's l1: 0.00457778
[1388]	valid_0's l1: 0.00454936
[1389]	valid_0's l1: 0.00455284
[1390]	valid_0's l1: 0.00454631
[1391]	valid_0's l1: 0.00455087
[1392]	valid_0's l1: 0.00453685
[1393]	valid_0's l1: 0.00452758
[1394]	valid_0's l1: 0.00453367
[1395]	valid_0's l1: 0.00451884
[1396]	valid_0's l1: 0.00450372
[1397]	valid_0's l1: 0.00448945
[1398]	valid_0's l1: 0.00449368
[1399]	valid_0's l1: 0.00449799
[1400]	valid_0's l1: 0.00448292
[1401]	valid_0's l1: 0.00448713
[1402]	valid_0's l1: 0.00449212
[1403]	valid_0's l1: 0.00447237
[1404]	valid_0's l1: 0.00447693
[1405]	valid_0's l1: 0.0044513
[1406]	valid_0's l1: 0.0044555
[1407]	valid_0's l1: 0.00443988
[1408]	valid_0's l1: 0.00444432
[1409]	valid_0's l1: 0.00442888
[1410]	valid_0's l1: 0.00443542
[1411]	valid_0's l1: 0.00444019
[1412]	valid_0's l1: 0.00443076
[1413]	valid_0's l1: 0.00443013
[1414]	val

In [19]:
estado_m = model.predict(data_mail[features],num_iteration=model.best_iteration)

In [126]:
estado_m

array([9.99995989e-01, 9.99999017e-01, 9.98314477e-01, ...,
       6.84902268e-07, 6.23828965e-07, 6.80123075e-07])